In [11]:
#@title login google 帳號, 按左邊按鈕執行。之後，輸入組合, GO，輸入組合, GO, ...
from itertools import combinations, product, chain, permutations
from collections import Counter
import numpy as np
from random import sample

hand_names=["無", "對", "兩對", "三條", "順", "同花", "葫蘆", "鐵支", "同花順"]
suit_names=list("SHDC")
rank_names=list("9TJQKA")

HAND ={n:i for i, n in enumerate(hand_names)}
NUM_SUITS = len(suit_names)
suit_list = list(range(NUM_SUITS))
NUM_RANKS = len(rank_names)
rank_list = list(range(NUM_RANKS))


all_cards = list(product(suit_list, rank_list))
all_cards_6blank = all_cards+[None]*6
all_hand_combinations = list(chain(*[combinations(all_cards, i) for i in range(6)]))

def compute_hand(cards):
    ranks = [i for s,i in cards]
    NUM_DIFF_RANKS = len(set(ranks))
    NUM_CARDS = len(cards)    
    if NUM_DIFF_RANKS == NUM_CARDS: # no duplicate ranks
        sorted_ranks = tuple(sorted(ranks, reverse=True))
        if NUM_CARDS < 5:
            return (HAND['無'], sorted_ranks)        
        is_flush =  all(s==cards[0][0] for s,i in cards)            
        is_straight = sorted_ranks[0]==sorted_ranks[4]+4
        if is_straight:
            hand_name = "同花順" if is_flush else "順"
        else:
            hand_name = "同花" if is_flush else "無"            
        return (HAND[hand_name], sorted_ranks)
    # with duplicate ranks
    hist = Counter(ranks).most_common()
    kickers = tuple(sorted([i for i, n in hist if n==1], reverse=True))
    if NUM_DIFF_RANKS == len(kickers)+1: # 對、三條、四條
        hand_name = ['對', '三條', '鐵支'][hist[0][1]-2]        
        return (HAND[hand_name], (hist[0][0],)+kickers)
    assert NUM_DIFF_RANKS == len(kickers)+2
    if hist[0][1]==3: # 葫蘆
        assert hist[1][1]==2
        return (HAND['葫蘆'], (hist[0][0], hist[1][0])+kickers)
    # 兩對
    r1, r2 = hist[0][0], hist[1][0]
    if r1 < r2:
        r1, r2 = r2, r1
    return (HAND['兩對'], (r1, r2)+kickers)
computed_hands={}
for c in all_hand_combinations:
    computed_hands[c] = compute_hand(c)

def cards_repr(cards):
    s = " ".join(suit_names[s]+rank_names[r] for s, r in cards)
    return '['+s+']'
def hand_repr(hand):
    return hand_names[hand[0]]+" "+"".join(rank_names[r] for r in hand[1])
def str2cards(s, fill_none=0):
  s = s.rstrip().lstrip()
  while '  ' in s:
    s = s.replace('  ', ' ')
  c = [None if x=='N' else (suit_names.index(x[0]), rank_names.index(x[1])) for x in s.split(' ')]
  if fill_none and len(c)<fill_none:
    c = c +[None]*(fill_none-len(c))
  return c
def str2setting(s):
  s=s.replace('[','').replace(']','\n')
  setting = [str2cards(x, fill_none=6) for x in  s.splitlines() if len(x)>1]
  assert len(setting)==5
  assert all(len(x)==6 for x in setting)
  return setting
def setting_repr(setting):
  return " ".join(cards_repr([x for x in s if x is not None]) for s in setting)

def random_setting():
  setting = sample(all_cards_6blank, 30)
  setting = [setting[i:i+6] for i in range(0,30,6)]
  return setting

def rec(result, setting, prob=1., cards=[]):
    if len(setting)==0:
        cards = tuple(sorted(cards))
        hand = computed_hands[cards]
        result[hand] = result.get(hand, 0) + prob
        return result
    #print(setting)
    sides = setting[0]
    assert len(sides)==6
    good_sides = [s for s in sides if s is not None]
    none_prob = (6-len(good_sides))/6
    other_prob = prob/6*(1+none_prob+none_prob**2)
    rec(result, setting[1:], prob * none_prob**3, cards)
    for s in good_sides:
        rec(result, setting[1:], other_prob, cards+[s])
    return result

def compute_dist(setting):
    result = rec({}, setting)
    result = sorted(result.items())
    a = 0
    final_result = []
    for h, p in result:
        final_result.append((h, p, a))
        a += p
    return final_result
def compare_dist(dist1, dist2):
  iter_dist2 = iter(dist2)
  w1, e = 0,0
  h2 = tuple()
  #h2 = -1
  for h1, p1, a1 in dist1:
      try:
          while h2 < h1:
              h2, p2, a2 = next(iter_dist2)                
          #print('x', h1, h2, p1, p2, e)
          if h2 == h1:
              e += p1*p2
              #print(h1, h2, p1, p2, e)
          w1 += p1*a2
      except StopIteration:
          w1 += p1
  w2 = 1- e - w1
  return w1, e, w2
def compare_setting(setting1, setting2):
  dist1 = compute_dist(setting1)
  dist2 = compute_dist(setting2)
  return compare_dist(dist1, dist2)
def setting_summary(setting):
  #print(setting_repr(setting))
  dist = compute_dist(setting)
  for name in hand_names:
    prob = sum(p for h,p,a in dist if h[0]==HAND[name])
    print(f"{name:}:\t{prob*100: >5.2f}%")

import ipywidgets as iw
from IPython.display import display, clear_output
setting1_str = """SA HT CJ DQ S9
SK HA CT DJ H9
SQ HK CA DT C9
SJ HQ CK DA D9
ST HJ CQ DK
"""
# 第二種組合
setting2_str = """SA HA CA DA S9 H9
SK HK CK DK
SQ HQ CQ DQ
SJ HJ CJ DJ
ST HT CT DT D9 C9
"""
import sys
def on_button_clicked(a):
  try:
    setting1 = str2setting(s1.value)
    setting2 = str2setting(s2.value)
  except AssertionError:
    with output2:
      clear_output()
    with output1:
       clear_output()
       print("組合的數量不正確")
    return
  except:
    with output2:
      clear_output()
      print(sys.exc_info()[1])
    with output1:
       clear_output()
       print("組合文字格式不正確:", sys.exc_info()[0])
    return
  try:
    w1, e, w2 = compare_setting(setting1, setting2)
  except:
    with output2:
      clear_output()
      print(sys.exc_info()[1])
    with output1:
       clear_output()
       print("可能有重複的牌:", sys.exc_info()[0])
    return

  output0.value =(f"第一種贏:{w1*100:.3f}%  平手:{e*100:.3f}%  第二種贏:{w2*100:.3f}%")
  with output1:
    clear_output()
    print(' '*40)
    setting_summary(setting1)
  with output2:
    clear_output()
    print(' '*40)
    setting_summary(setting2)
output0 = iw.Label(cols=60)
output0.style.text_color='red'
output1 = iw.Output(rows=10, cols=40)
output2 = iw.Output(rows=10, cols=40)

s1 = iw.Textarea(rows=5, width="50%", value=setting1_str)
s2 = iw.Textarea(rows=5, width="50%", value=setting2_str)
button = iw.Button(description='GO')
button.on_click(on_button_clicked)
display(
    iw.VBox([iw.HBox([s1, s2]),
             button,
             output0,
             iw.HBox([output1, output2])
             ])
)